Learned Concepts:

1- What is a computation graph?

2- What are symbolic variables?

3- How to create a function out of the computation graph?\n

4- How to evaluate the function with real values?

In [50]:
import numpy as np
import theano.tensor as T
import theano

# Symbolic Variables 
x= T.iscalar('x')
y= T.iscalar('y')
z= T.iscalar('z')
a= T.iscalar('a')

# Construct computation Graph
x= y+z
a= x**2

# Compile the function
f= theano.function([y,z], a)

# Create real values and evaluate the function
y_real= 10
z_real= 20

a_real= f(y_real, z_real)

print('Output is ', a_real)

('Output is ', array(900, dtype=int32))


Computation Graph Exercise:
Create a function that computes the logistic function

In [51]:

# Write your code here


Learned Concepts:
1- Shared Variables 
2- Updates

In [52]:
state= theano.shared(0.0)
inc= T.iscalar('inc')
accumulator= theano.function([inc], state, updates= [( state, state+inc)])
incc= 5
print('State = ', accumulator(incc))
print('State = ', accumulator(incc))
print('State = ', accumulator(incc))
print('State = ', accumulator(incc))

('State = ', array(0.0))
('State = ', array(5.0))
('State = ', array(10.0))
('State = ', array(15.0))


Logistic Regression Tutorial:

This uses the shared variables, copmmutation graph, updates concept

In [53]:
N = 400                                   # training sample size
feats = 784                               # number of input variables

# generate a dataset: D = (input_values, target_class)
D = (np.random.randn(N, feats), np.random.randint(size=N, low=0, high=2))
training_steps = 10000

# Declare Theano symbolic variables
x = T.dmatrix("x")
y = T.dvector("y")

# initialize the weight vector w randomly
#
# this and the following bias variable b
# are shared so they keep their values
# between training iterations (updates)
w = theano.shared(np.random.randn(feats), name="w")

# initialize the bias term
b = theano.shared(0., name="b")

print("Initial model:")
#print(w.get_value())
print(b.get_value())

Initial model:
0.0


In [54]:
# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability that target = 1
prediction = p_1 > 0.5                    # The prediction thresholded
cent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = cent.mean() + 0.01 * (w ** 2).sum()# The cost to minimize

gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # w.r.t weight vector w and
                                          # bias term b
                                          # (we shall return to this in a
                                          # following section of this tutorial)

# Compile
train = theano.function(
          inputs= [x,y],
          outputs= [prediction, cent],
          updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)))
predict = theano.function(inputs=[x], outputs=prediction)

In [55]:
# Train
for i in range(training_steps):
    pred, err = train(D[0], D[1])

print("Final model:")
#print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))

Final model:
0.206249806352
target values for D:
[1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 1 0 1 1 0 0 0 1 0 0 1 1 0 0 0 0 1 1 1 0 0
 1 0 1 0 0 1 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 1 0 1 0 0 0 1 0 1 0 0 1 0 0
 0 0 1 1 1 1 0 0 0 0 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 0 0 0
 0 1 0 0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 0 0 1 0 1 0 1 1 0 1 1 1 1 0 1 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0
 1 1 1 1 0 0 1 1 0 1 1 1 1 0 0 1 1 1 1 0 0 0 1 0 0 1 0 1 0 0 0 1 1 0 1 1 1
 1 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 1 0 1 1 1 0 0 1 1 0 0 0 1 1 1 0 1 1 1
 1 1 1 0 1 1 0 0 1 0 0 1 0 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1
 0 1 0 1 1 1 0 0 1 0 0 1 1 1 0 0 1 1 1 1 1 0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 0
 0 1 1 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1
 1 1 0 1 1 0 1 1 0 1 0 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 0 0 1 1]
prediction on D:
[1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 1 0 1 1 0 0 0 1 0 0 1 1 0 0 0 0 1 1 1 0 0
 1 0 1 0 0 1 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 

Exercise Write your own simple neural network (Fully Connected one). 

It has two layers: 

Layer1: y1= tanh(w1*x+b1)

Layer2: y2= w2*y1+b2

Layer3: y3= softmax(y2)

Cost is categorical cross entropy


In [56]:
# 1- Create shared Variables for weights and initialize them to random + symbolic variables
N, D, H, C= 10, 1000, 100, 1

x= T.matrix('x')
y= T.vector('y', dtype= 'int64')

w1= theano.shared(np.random.randn(D, H),name='w1')
w2= theano.shared(np.random.randn(H, C),name='w2')
b1= theano.shared(np.zeros((H,)), name= 'b1')
b2= theano.shared(np.zeros((C,)), name= 'b2')

# 2- Forward Pass Computation Graph
# Write Code Here

# 3- Backward Pass Compute Gradients
# Write Code Here

# 4- Define your train function with the proper updates using sgd
# Write Code Here

# 5- Evaluate your train function
xx= np.random.randn(N, D)
yy= np.random.randint(size= N, low= 0.0, high= C+1)

# Write Code Here


Solution to the Simple Neural Network Example

In [57]:
import numpy as np
import theano.tensor as T
import theano
# 1- Create shared Variables for weights and initialize them to random + symbolic variables
N, D, H, C= 10, 1000, 100, 1

x= T.matrix('x')
y= T.vector('y', dtype= 'int64')

w1= theano.shared(np.random.randn(D, H),name='w1')
w2= theano.shared(np.random.randn(H, C),name='w2')
b1= theano.shared(np.zeros((H,)), name= 'b1')
b2= theano.shared(np.zeros((C,)), name= 'b2')

# 2- Forward Pass Computation Graph
y1= T.tanh(x.dot(w1)+b1)
y2= y1.dot(w2)+b2
p_1 = 1 / (1 + T.exp(-y2))   # Probability that target = 1
prediction = p_1 > 0.5                    # The prediction thresholded
cent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = cent.mean()

# 3- Backward Pass Compute Gradients
dw1, dw2, db1, db2= T.grad(cost, [w1, w2, b1, b2])

# 4- Define your train function with the proper updates using sgd
lr= 0.001
train= theano.function(inputs=[x,y],
                       outputs=[cost, prediction],
                       updates= [(w1, w1-lr*dw1), (w2, w2-lr*dw2), (b1, b1-lr*db1), (b2, b2-lr*db2)])

# 5- Evaluate your train function
xx= np.array(np.random.randn(N, D), dtype="float32")
yy= np.random.randint(size= N, low= 0.0, high= C+1)

iters=100
for i in range(iters):
    for j in range(N):
        cost, preds = train(xx[j, :].reshape((1,D)),yy[j].reshape((1,)) )
        if i == iters-1:
            print('Target Labels are hiii', yy[j])
            print('Current predictions are ', preds)

('Target Labels are hiii', 1)
('Current predictions are ', array([[1]], dtype=int8))
('Target Labels are hiii', 0)
('Current predictions are ', array([[0]], dtype=int8))
('Target Labels are hiii', 1)
('Current predictions are ', array([[1]], dtype=int8))
('Target Labels are hiii', 1)
('Current predictions are ', array([[0]], dtype=int8))
('Target Labels are hiii', 1)
('Current predictions are ', array([[1]], dtype=int8))
('Target Labels are hiii', 0)
('Current predictions are ', array([[0]], dtype=int8))
('Target Labels are hiii', 1)
('Current predictions are ', array([[1]], dtype=int8))
('Target Labels are hiii', 0)
('Current predictions are ', array([[0]], dtype=int8))
('Target Labels are hiii', 0)
('Current predictions are ', array([[0]], dtype=int8))
('Target Labels are hiii', 1)
('Current predictions are ', array([[1]], dtype=int8))


In [58]:
class network():
    def __init__ (self):
        self.params = {}
        lr = np.array(0.001, dtype="float32")

        self.x = T.vector('x')
        self.y = T.scalar('y')
                                                                                                                                                                       
        self.RNN_out = self.RNN(self.x)
        self.loss = self.euc_loss(self.RNN(self.x), self.y)
        
        
        self.grads = T.grad(self.loss, wrt = list(self.params.values()) )

        gshared = [theano.shared(p.get_value() * np.array(0.0, dtype="float32"), name='%s_grad' % k) for k, p in self.params.items()]
        
        gsup = [(gs, g) for gs, g in zip(gshared, self.grads)]
        pup = [(param, param - lr*g) for param, g in zip(self.params.values(), gshared)]
        self.f_forward = theano.function([self.x], outputs=[self.RNN_out])
        
        self.train = theano.function([self.x, self.y], outputs=[self.RNN_out, self.loss], updates=gsup + pup)
    
        
    def RNN(self, X):
        self.params['W'] = theano.shared(value=np.array(np.random.rand(), dtype="float32"), name= 'W')
        self.params['U'] = theano.shared(value=np.array(np.random.rand(), dtype="float32"), name= 'U')
        self.params['b'] = theano.shared(value=np.array(np.random.rand(), dtype="float32"), name= 'b')
        self.params['Wo'] = theano.shared(value=np.array(np.random.rand(), dtype="float32"), name= 'Wo')
        
       
        n_steps = X.shape[0]
        def step(x, _h): 
            h = T.tanh(self.params['W']*_h + self.params['U']*x + self.params['b'])
            y = self.params['Wo'] * h

            return y, h
            

        results, update = theano.scan(step, sequences = X, outputs_info =[None,  np.array(0.0, dtype="float32")], n_steps = n_steps)
        return results[0][-1]
            

    def euc_loss(self, inp, label):
        return (inp-label)**2

def dummy(x):
    return x.sum()

net = network()
max_iter = 100000
disp_freq = 2000

loss = np.zeros(disp_freq)
for i in range(max_iter):
    x = np.array(np.random.rand(2), dtype="float32")
    y = dummy(x)
    loss[i%disp_freq] = net.train(x,y)[1]
    if i%disp_freq == 0:
        print ("loss at iter%s = %s"%(i,loss.mean()))

loss at iter0 = 0.000105520702899
loss at iter2000 = 0.178741631224
loss at iter4000 = 0.119768433419
loss at iter6000 = 0.104258098036
loss at iter8000 = 0.0923206752571
loss at iter10000 = 0.0847378559605
loss at iter12000 = 0.0687221718054
loss at iter14000 = 0.0552323171445
loss at iter16000 = 0.0360805221671
loss at iter18000 = 0.0244861322183
loss at iter20000 = 0.0183551333875
loss at iter22000 = 0.0145539047568
loss at iter24000 = 0.0121873505553
loss at iter26000 = 0.0103094233416
loss at iter28000 = 0.00955641794375
loss at iter30000 = 0.00903140547499
loss at iter32000 = 0.00814901075978
loss at iter34000 = 0.00665179944574
loss at iter36000 = 0.00679499831116
loss at iter38000 = 0.00665677671406
loss at iter40000 = 0.00593863845263
loss at iter42000 = 0.00561237437179
loss at iter44000 = 0.00479485890752
loss at iter46000 = 0.00509261138187
loss at iter48000 = 0.00468719436816
loss at iter50000 = 0.0044201458915
loss at iter52000 = 0.00439219409057
loss at iter54000 = 0.004